In [10]:
import os
import time
import json
import pandas as pd
from gerbil_api_wrapper.gerbil import Gerbil

In [35]:
def read_json(filename):
    with open(filename) as f:
        return json.load(f)
    
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def extract_string_qald(json_data, language):
    for question in json_data['question']:
        if question['language'] == language:
            return question['string']
        
standard_columns = ['system', 'dataset', 'language', 'id', 'experiment_url']

### Original

In [25]:
systems = ['tebaqa']
datasets = ['qald-9-plus-test-dbpedia', 'qald-9-plus-train-dbpedia']
languages = ['en']

In [26]:
for system in systems:
    for dataset in datasets:
        for language in languages:
            data = read_json(f"../original_data/original-{system}-{dataset}-{language}.json")

            for question in data:

                id_ = question["id"]

                answers_pred = question["answers_pred"]
                if type(answers_pred) == dict:
                    answers_pred_formatted = [{"x": {"type": "uri", "value": value}} for value in answers_pred['answer']]
                else:
                    answers_pred_formatted = []

                string = extract_string_qald(question, 'en') # here we use english question to get the string

                qald_template_true = {
                    "questions": [ question ]
                }

                qald_template_pred = {
                    "questions": [{
                        "id": f"{id_}",
                        "question": [{
                            "language": "en",
                            "string": string
                        }],
                        "query": {
                            "sparql": ""
                        },
                        "answers": [{
                            'head': {
                                    'vars': ['x']
                                    },
                            'results': {
                                'bindings': answers_pred_formatted
                            }
                        }]
                    }]
                }
                
                # check if the file exists
                if os.path.isfile(f"../original_data/{system}-{dataset}-{language}.log"):
                    df = pd.read_csv(f"../original_data/{system}-{dataset}-{language}.log", sep='\t')
                    if "id" not in df.columns:
                        df.columns = standard_columns
                    if id_ in df['id'].values: 
                        print(f"Skipping {id_}")
                        continue  # TODO: doesn't work for some reason CHECK!!!

                write_json(qald_template_true, f"../original_data/tmp/{system}-{dataset}-{id_}-{language}-true.json")
                write_json(qald_template_pred, f"../original_data/tmp/{system}-{dataset}-{id_}-{language}-pred.json")

                wrapper = Gerbil(
                    gold_standard_file=f"../original_data/tmp/{system}-{dataset}-{id_}-{language}-true.json",
                    test_results_file=f"../original_data/tmp/{system}-{dataset}-{id_}-{language}-pred.json",
                    test_results_name=f"{system}-{dataset}-{id_}-{language}-pred",
                    gold_standard_name=f"{system}-{dataset}-{id_}-{language}-true",
                    language=language,
                    # check_for_another_experiments=True
                )

                experiment_url = str(wrapper.get_experiment_url + wrapper.experiment_id)

                with open(f"../original_data/{system}-{dataset}-{language}.log", 'a') as f:
                    f.write(f"{system}\t{dataset}\t{language}\t{id_}\t{experiment_url}\n")
                
                time.sleep(0.1)

2023-05-31 13:39:42,629 - 
Request to: https://gerbil-qa.aksw.org/gerbil/file/upload
Data: {'name': 'tebaqa-qald-9-plus-test-dbpedia-99-en-true', 'multiselect': None}
Files: {'file': ('tebaqa-qald-9-plus-test-dbpedia-99-en-true', <_io.BufferedReader name='../original_data/tmp/tebaqa-qald-9-plus-test-dbpedia-99-en-true.json'>, 'application/json')}
2023-05-31 13:39:42,711 - Got response: {'files': [{'name': 'tebaqa-qald-9-plus-test-dbpedia-99-en-true', 'size': '3Kb', 'url': None, 'thumbnailUrl': None, 'deleteUrl': None, 'deleteType': None, 'error': None, 'fileType': 'application/json'}]} (200)
2023-05-31 13:39:42,712 - 
Request to: https://gerbil-qa.aksw.org/gerbil/file/upload
Data: {'name': 'tebaqa-qald-9-plus-test-dbpedia-99-en-pred', 'multiselect': None}
Files: {'file': ('tebaqa-qald-9-plus-test-dbpedia-99-en-pred', <_io.BufferedReader name='../original_data/tmp/tebaqa-qald-9-plus-test-dbpedia-99-en-pred.json'>, 'application/json')}
2023-05-31 13:39:42,794 - Got response: {'files': [{

### Translations

In [ ]:
data_files = [file for file in os.listdir("../translated_data/") if system in file.lower() and dataset.replace('-', '') in file.lower() and language in file.lower()]
data_files

In [ ]:
for data_file in data_files:
    data = read_json(f"../translated_data/{data_file}")["questions"]
    for question in data:

        id_ = question["id"]

        answers_pred = question["answers_pred"]
        if type(answers_pred) == dict:
            answers_pred_formatted = [{"x": {"type": "uri", "value": value}} for value in answers_pred['answer']]
        else:
            answers_pred_formatted = []

        string = extract_string_qald(question, 'en') # here we use english question to get the string

        qald_template_true = {
            "questions": [ question ]
        }

        qald_template_pred = {
            "questions": [{
                "id": f"{id_}",
                "question": [{
                    "language": "en",
                    "string": string
                }],
                "query": {
                    "sparql": ""
                },
                "answers": [{
                    'head': {
                            'vars': ['x']
                            },
                    'results': {
                        'bindings': answers_pred_formatted
                    }
                }]
            }]
        }
        
        write_json(qald_template_true, f"../translated_data/tmp/{data_file}-{id_}-true.json")
        write_json(qald_template_pred, f"../translated_data/tmp/{data_file}-{id_}-pred.json")

        wrapper = Gerbil(
            gold_standard_file=f"../translated_data/tmp/{data_file}-{id_}-true.json",
            test_results_file=f"../translated_data/tmp/{data_file}-{id_}-pred.json",
            test_results_name=f"{data_file}-{id_}-pred",
            gold_standard_name=f"{data_file}-{id_}-true",
            language=language,
            # check_for_another_experiments=True
        )

        experiment_url = str(wrapper.get_experiment_url + wrapper.experiment_id)

        # check if the file exists
        if os.path.isfile(f"../translated_data/{data_file}.log"):
            df = pd.read_csv(f"../translated_data/{data_file}.log", sep='\t', header=None)
            df.columns = ['system', 'dataset', 'language', 'id', 'experiment_url']
            if id_ in df['id'].values:
                print(f"Skipping {id_}")
                continue  

        with open(f"../translated_data/{data_file}.log", 'a') as f:
            f.write(f"{system}\t{dataset}\t{language}\t{id_}\t{experiment_url}\n")
        
        time.sleep(0.5)
        # TODO: produce error with upload URL 

## Parse results

In [37]:
metrics = [
    "macroF1",
    "microF1",
    "macroPrecision",
    "microPrecision",
    "macroRecall",
    "microRecall",
    "Macro_F1_QALD"
]

In [38]:
dir_path = "../original_data/" # or "../translated_data/"
log_files = [file for file in os.listdir(dir_path) if ".log" in file] 

In [40]:
for log_file in log_files:
    df = pd.read_csv(f"{dir_path}{log_file}", sep='\t', header=None)
    if df.shape[1] == len(standard_columns):
        df.columns = standard_columns
    elif df.shape[1] == len(standard_columns) + len(metrics):
        continue
    else:
        print("ERROR, check", log_file)
        continue
    
    print(log_file, ":", df.shape)

    id_jsonld = dict()

    for i, row in df.iterrows():
        id_jsonld[row['id']] = Gerbil.get_results(row['experiment_url'])

    for metric in metrics:
        df[metric] = df['id'].apply(lambda x: float(id_jsonld[x]['@graph'][1][metric]))

    df.to_csv(f"{dir_path}{log_file}", sep='\t', index=False)

deeppavlov-qald-9-plus-train-wikidata-ru.log : (371, 5)
qanswer-qald-9-plus-train-wikidata-fr.log : (371, 5)
deeppavlov-qald-10-en.log : (395, 5)
qanswer-qald-9-plus-test-wikidata-ru.log : (136, 5)
qanswer-qald-9-plus-train-wikidata-en.log : (371, 5)
qanswer-qald-10-zh.log : (394, 5)
qanswer-qald-9-plus-train-wikidata-de.log : (371, 5)
deeppavlov-qald-9-plus-train-wikidata-en.log : (371, 5)
deeppavlov-qald-9-plus-test-wikidata-en.log : (136, 5)
deeppavlov-qald-9-plus-test-wikidata-ru.log : (136, 5)
qanswer-qald-10-de.log : (394, 5)
qanswer-qald-9-plus-test-wikidata-en.log : (136, 5)
qanswer-qald-9-plus-test-wikidata-fr.log : (136, 5)
tebaqa-qald-9-plus-train-dbpedia-en.log : (408, 5)
qanswer-qald-9-plus-test-wikidata-de.log : (136, 5)
qanswer-qald-10-ru.log : (394, 5)
qanswer-qald-9-plus-train-wikidata-ru.log : (371, 5)
tebaqa-qald-9-plus-test-dbpedia-en.log : (150, 5)


## Compare results

In [25]:
df_1 = pd.read_csv(f"../translated_data/deeppavlov-QALD10Train-MT_ru-no_ner-no_el-nllb_mt.json.json.log", sep='\t')
df_2 = pd.read_csv(f"../translated_data/deeppavlov-QALD10Train-MT_ru-spacy_ner-mgenre_el-mbart_mt.json.json.log", sep='\t')
df_3 = pd.read_csv(f"../original_data/deeppavlov-qald-10-ru.log", sep='\t')

In [26]:
df_1.macroF1.mean(), df_2.macroF1.mean(), df_3.macroF1.mean()

(0.31109536261820525, 0.32445228206649523, 0.18900169204737732)